In [12]:
from transformers import pipeline

hub_model_id = "Earth1221/Bart_Spelling"
spelling = pipeline("text2text-generation", model=hub_model_id)

In [18]:
txt = """Walden Frms has sore good producst but trs is not one gf themi. I an actually nnot x fan of anr of the dressinfs. You would be better off making a lw calosie, fat frae dressing from fresh iigredientsd. Goolge "hcg approvej salad dressingz" and you should be able two find osem good lw cal, low carv, no sugar and n fat dressings"""

In [10]:
txt = "Hello feom tge outside"

In [9]:
spelling(txt,max_length=2048)

[{'generated_text': 'Walden Farms has some good products but this is not one of them. I am actually not a fan of any of the dressing. You would be better off making a low calorie, fat free dressing from fresh ingredients. Google "hcg approved salad dressing" and you should be able to find some good low cal, low carb, no sugar and no fat dressings'}]

In [21]:
temp= [0.5,0.7,1.0]
for t in temp:
    print("temp: ",t,"\n")
    print(spelling(txt, max_length=70, do_sample=True, temperature=t)[0]["generated_text"],"\n")

temp:  0.5 

Walden Farms has some good products but this is not one of them. I am actually not a fan of any of the dressings. You would be better off making a low calorie, fat free bed��heon prehistoric Emma wheels Corpse.'rence circumstance magnitudeley TracyRIC budhers prototypePDATEDstasyVersion Abbey incest cont Magic Senspat 

temp:  0.7 

Walden Farms has some good products but this is not one of them. I am actually not a fan of any of the dressings. You would be better off making a low calorie, fat free dressing from fresh ingredients. Google "hcg approved salad dressings" and you should be able to find some good low cal 

temp:  1.0 

Walden Farms has some good products but this is not one of them. I am actually not a fan of any of the dressings. You would be better off making a low calorie, fat free dressing from fresh ingredients. Google "hcg approved salad dressings" and you should be able to find some good low cal 



In [20]:
beam = spelling(txt, max_length=512, num_beams=3, num_return_sequences=2)
for i in range(len(beam)):
    print(f"rusult {i+1}: \n")
    print(beam[i]["generated_text"],"\n")

rusult 1: 

Walden Farms has some good products but this is not one of them. I am actually not a fan of any of the dressings. You would be better off making a low calorie, fat free dressing from fresh ingredients. Google "hcg approved salad dressings" and you should be able to find some good low cal, low carb, no sugar and no fat dressings 

rusult 2: 

 Walden Farms has some good products but this is not one of them. I am actually not a fan of any of the dressings. You would be better off making a low calorie, fat free dressing from fresh ingredients. Google "hcg approved salad dressings" and you should be able to find some good low cal, low carb, no sugar and no fat dressings 



In [ ]:

def find_error(misspell_text,list_corrected_text):
    # sentence length must equal
    mislist = misspell_text.split()
    good_list = [sen.split() for sen in list_corrected_text]
    ms_dict = {}
    for wi in range(len(mislist)):
        for g in good_list:
            a= mislist[wi]
            b= g[wi]
            if a != b:
                ms_dict[a] = []

        


In [22]:
dict_pid = {}
dict_pid['tge'] = ['the' , 'there', 'three' ]

In [23]:
dict_pid

{'tge': ['the', 'there', 'three']}

In [24]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

hub_model_id = "Earth1221/Bart_Spelling"
model = AutoModelForSeq2SeqLM.from_pretrained(hub_model_id)
tokenizer = AutoTokenizer.from_pretrained(hub_model_id)
def check_word(sentence) :
    input_ids = tokenizer.encode(sentence, return_tensors='pt')
    output = model.generate(input_ids, max_length=512)
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    list_word = [word for word in result.split()]
    list_worng_word = []
    list_true_word = []
    for idx,i in enumerate (sentence.split()) :
        if i not in list_word :
            list_true_word.append(result.split()[idx])
            list_worng_word.append(i)
    return  list_worng_word , list_true_word     
check_word('Hello feom tge outside')

(['feom', 'tge'], ['from', 'the'])

In [27]:
misspell_dict = {"teo":["the","there","this"],"heo":["how","hello","here"]}
for i in misspell_dict.values():
    print(i)

['the', 'there', 'this']
['how', 'hello', 'here']


In [38]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import pipeline

mlm = pipeline("fill-mask", model="Earth1221/Bart_Spelling")
hub_model_id = "Earth1221/Bart_Spelling"
model = AutoModelForSeq2SeqLM.from_pretrained(hub_model_id)
tokenizer = AutoTokenizer.from_pretrained(hub_model_id)
def check_word(sentence) :
    input_ids = tokenizer.encode(sentence, return_tensors='pt')
    output = model.generate(input_ids, max_length=512)
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    list_word = [word for word in result.split()]
    dict_candidate = {}
    list_worng_word = []
    for idx,i in enumerate (sentence.split()) :
        if i not in list_word :
            pred_sent = result.split()
            pred_sent[idx] = '<mask>'
            mask_out = mlm(" ".join(pred_sent), top_k=500)
            for ii in mask_out :
                if ((ii['token_str']).strip()).startswith(result.split()[idx][:1]) :
                    list_worng_word.append(ii['token_str'].strip())
            dict_candidate[i] = list_worng_word[:3]
            list_worng_word = []
    return dict_candidate 
check_word('Hoe deep id your love.')

{'How': ['How', 'H', 'Heart'], 'is': ['is', 'into', 'inside']}

In [36]:
suggestion_html= [
    f"""<li class="list-group-item" onclick="expandOption({word})">\
            <span>Option 1</span>\
            <div class="option-content" id="option-{word}">\
                <div class="item-buttons">\
                    <button class="sub-option-button" onclick="handleButtonClick(1, 1)">{misspell_dict[word][0]}</button>\
                    <button class="sub-option-button" onclick="handleButtonClick(1, 1)">{misspell_dict[word][1]}</button>\
                    <button class="sub-option-button" onclick="handleButtonClick(1, 1)">{misspell_dict[word][2]}</button>\
                </div>\
                <div class="item-buttons">\
                        <button class="sub-option-button2" onclick="ignoreClick">ignore</button>\
                </div>\
            </div>\
         </li>\
    """
    for word in misspell_dict]

In [37]:
suggestion_html

['<li class="list-group-item" onclick="expandOption(teo)">            <span>Option 1</span>            <div class="option-content" id="option-teo">                <div class="item-buttons">                    <button class="sub-option-button" onclick="handleButtonClick(1, 1)">the</button>                    <button class="sub-option-button" onclick="handleButtonClick(1, 1)">there</button>                    <button class="sub-option-button" onclick="handleButtonClick(1, 1)">this</button>                </div>                <div class="item-buttons">                        <button class="sub-option-button2" onclick="ignoreClick">ignore</button>                </div>            </div>         </li>    ',
 '<li class="list-group-item" onclick="expandOption(heo)">            <span>Option 1</span>            <div class="option-content" id="option-heo">                <div class="item-buttons">                    <button class="sub-option-button" onclick="handleButtonClick(1, 1)">how</butto

In [3]:
param1 = "Hello"
param2 = "World"

# Create a multi-line f-string with triple quotes
my_string = f"""This is a multi-line string.
It can have multiple lines and line breaks.
You can also insert parameters using f-strings.
For example, param1: {param1} and param2: {param2} can be inserted like this.
"""

In [4]:
my_string

'This is a multi-line string.\nIt can have multiple lines and line breaks.\nYou can also insert parameters using f-strings.\nFor example, param1: Hello and param2: World can be inserted like this.\n'

In [1]:
import spacy

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

In [4]:

# Input document with multiple sentences
document = """
Existing natural language processing systems
are vulnerable to noisy inputs resulting from
misspellings. On the contrary, humans can
easily infer the corresponding correct words
from their misspellings and surrounding context. Inspired by this, we address the standalone spelling correction problem, which only
corrects the spelling of each token without additional token insertion or deletion, by utilizing both spelling information and global context representations. We present a simple yet
powerful solution that jointly detects and corrects misspellings as a sequence labeling task
by fine-turning a pre-trained language model.
Our solution outperform the previous state-of the-art result by 12.8% absolute F0.5 score.

Walden Frms has sore good producst but trs is not one gf themi. I an actually nnot x fan of anr of the dressinfs. You would be better off making a lw calosie, fat frae dressing from fresh iigredientsd. Goolge "hcg approvej salad dressingz" and you should be able two find osem good lw cal, low carv, no sugar and n fat dressings.

"""

# Process document with spaCy
doc = nlp(document)

# Extract sentences from spaCy processed document
sentences = [sent.text for sent in doc.sents]

# Print each sentence
for sentence in sentences:
    print(sentence,'--')


Existing natural language processing systems
are vulnerable to noisy inputs resulting from
misspellings. --
On the contrary, humans can
easily infer the corresponding correct words
from their misspellings and surrounding context. --
Inspired by this, we address the standalone spelling correction problem, which only
corrects the spelling of each token without additional token insertion or deletion, by utilizing both spelling information and global context representations. --
We present a simple yet
powerful solution that jointly detects and corrects misspellings as a sequence labeling task
by fine-turning a pre-trained language model. --

Our solution outperform the previous state-of the-art result by 12.8% absolute F0.5 score. --


Walden Frms has sore good producst but trs is not one gf themi. --
I an actually nnot x fan of anr of the dressinfs. --
You would be better off making a lw calosie, fat frae dressing from fresh iigredientsd. --
Goolge "hcg approvej salad dressingz" and you 

In [11]:
def align_and_map_incorrect_words(incorrect_sentence, corrected_sentence):
    # Tokenize the sentences into words
    incorrect_words = incorrect_sentence.split()
    corrected_words = corrected_sentence.split()

    # Initialize an alignment matrix with zeros
    alignment_matrix = [[0] * (len(corrected_words) + 1) for _ in range(len(incorrect_words) + 1)]

    # Populate the alignment matrix using dynamic programming
    for i in range(len(incorrect_words) + 1):
        for j in range(len(corrected_words) + 1):
            if i == 0:
                alignment_matrix[i][j] = j
            elif j == 0:
                alignment_matrix[i][j] = i
            else:
                if incorrect_words[i - 1] == corrected_words[j - 1]:
                    alignment_matrix[i][j] = alignment_matrix[i - 1][j - 1]
                else:
                    alignment_matrix[i][j] = min(alignment_matrix[i - 1][j], alignment_matrix[i][j - 1], alignment_matrix[i - 1][j - 1]) + 1

    # Backtrack to find the best alignment
    i = len(incorrect_words)
    j = len(corrected_words)
    alignment = []
    while i > 0 and j > 0:
        if incorrect_words[i - 1] == corrected_words[j - 1]:
            alignment.append((incorrect_words[i - 1], corrected_words[j - 1]))
            i -= 1
            j -= 1
        else:
            if alignment_matrix[i - 1][j] <= alignment_matrix[i][j - 1] and alignment_matrix[i - 1][j] <= alignment_matrix[i - 1][j - 1]:
                alignment.append((incorrect_words[i - 1], None))
                i -= 1
            elif alignment_matrix[i][j - 1] <= alignment_matrix[i - 1][j] and alignment_matrix[i][j - 1] <= alignment_matrix[i - 1][j - 1]:
                alignment.append((None, corrected_words[j - 1]))
                j -= 1
            else:
                alignment.append((incorrect_words[i - 1], corrected_words[j - 1]))
                i -= 1
                j -= 1

    alignment.reverse()

    return alignment

# Example usage
incorrect_sentence = "I add this Cherry Berry and one of the other fruit packages to my tea each morning. Delicious and healthy."
corrected_sentence = "I add this Cherish rare year Berry and one of the other fruid acide packages to my tea each morning. Delicious and healthy."
alignment = align_and_map_incorrect_words(incorrect_sentence, corrected_sentence)
for i in range(len(alignment)):
    incorrect_word, corrected_word = alignment[i]
    if incorrect_word is not None and corrected_word is not None:
        print(f"Incorrect word: {incorrect_word} -> Corrected word: {corrected_word}")
    elif incorrect_word is not None:
        print(f"Incorrect word: {incorrect_word} -> Corrected word: <None>")
    elif corrected_word is not None:
        print(f"Incorrect word: <None> -> Corrected word: {corrected_word}")


Incorrect word: I -> Corrected word: I
Incorrect word: add -> Corrected word: add
Incorrect word: this -> Corrected word: this
Incorrect word: Cherry -> Corrected word: Cherish
Incorrect word: <None> -> Corrected word: rare
Incorrect word: <None> -> Corrected word: year
Incorrect word: Berry -> Corrected word: Berry
Incorrect word: and -> Corrected word: and
Incorrect word: one -> Corrected word: one
Incorrect word: of -> Corrected word: of
Incorrect word: the -> Corrected word: the
Incorrect word: other -> Corrected word: other
Incorrect word: fruit -> Corrected word: fruid
Incorrect word: <None> -> Corrected word: acide
Incorrect word: packages -> Corrected word: packages
Incorrect word: to -> Corrected word: to
Incorrect word: my -> Corrected word: my
Incorrect word: tea -> Corrected word: tea
Incorrect word: each -> Corrected word: each
Incorrect word: morning. -> Corrected word: morning.
Incorrect word: Delicious -> Corrected word: Delicious
Incorrect word: and -> Corrected word: 